In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data/1091402.txt", delim_whitespace=True, skiprows=[1], na_values=["-9999"])
data = data.reset_index()
data = data[["DATE", "PRCP", "TMAX", "TMIN"]]
data.head()

,DATE,PRCP,TMAX,TMIN
0,19520101,0.31,39.0,34.0
1,19520102,NaN,37.0,34.0
2,19520103,0.14,36.0,NaN
3,19520104,0.05,30.0,25.0
4,19520105,0.06,30.0,25.0


In [3]:
data = data.dropna()
data["TMAX"] = data["TMAX"].round(0).astype(int)
data["TMIN"] = data["TMIN"].round(0).astype(int)
data["TAVG"] = (data["TMAX"] + data["TMIN"])/2
data["TEMP_C"] = (data['TAVG'] -32) / 1.8
data.head()


,DATE,PRCP,TMAX,TMIN,TAVG,TEMP_C
0,19520101,0.31,39,34,36.5,2.500000
3,19520104,0.05,30,25,27.5,-2.500000
4,19520105,0.06,30,25,27.5,-2.500000
6,19520107,0.03,43,32,37.5,3.055556
8,19520109,0.18,39,32,35.5,1.944444


In [4]:
data["YEAR"] = data["DATE"].astype(str).str.slice(0,4).astype(int)
data["MONTH"] = data["DATE"].astype(str).str.slice(4,6).astype(int)
data["YEARMONTH"] = data["DATE"].astype(str).str.slice(0,6).astype(int)

data.head()
data.dtypes


DATE           int64
PRCP         float64
TMAX           int32
TMIN           int32
TAVG         float64
TEMP_C       float64
YEAR           int32
MONTH          int32
YEARMONTH      int32
dtype: object

In [5]:
reference_temps = data.groupby(by="MONTH").mean()

In [6]:
reference_temps = reference_temps.reset_index()
reference_temps = reference_temps[["MONTH", "TAVG", "TEMP_C"]]
reference_temps["ref_temp"] = reference_temps["TEMP_C"]
reference_temps = reference_temps.drop(columns=["TEMP_C"])
reference_temps.head()

,MONTH,TAVG,ref_temp
0,1,21.154787,-6.025118
1,2,20.899016,-6.167213
2,3,27.792892,-2.337282
3,4,38.546655,3.637031
4,5,50.077766,10.043203


In [7]:
temp_avg = data.groupby(by="YEARMONTH")[["TEMP_C", "MONTH"]].mean()
temp_avg = temp_avg.reset_index()
temp_avg.head()

,YEARMONTH,TEMP_C,MONTH
0,195201,-0.416667,1.0
1,195202,-3.638889,2.0
2,195203,-10.895062,3.0
3,195204,-0.277778,4.0
4,195205,5.555556,5.0


In [8]:
join = pd.merge(temp_avg, reference_temps, on="MONTH", how="left")
join.head()
join["DIFF"] = join["TEMP_C"] - join["ref_temp"]
join.head()

,YEARMONTH,TEMP_C,MONTH,TAVG,ref_temp,DIFF
0,195201,-0.416667,1.0,21.154787,-6.025118,5.608452
1,195202,-3.638889,2.0,20.899016,-6.167213,2.528324
2,195203,-10.895062,3.0,27.792892,-2.337282,-8.557779
3,195204,-0.277778,4.0,38.546655,3.637031,-3.914809
4,195205,5.555556,5.0,50.077766,10.043203,-4.487648


In [9]:
new_names = {"TEMP_C": "TEMP_C_Helsinki", "TAVG": "TAVG_Helsinki", "ref_temp": "ref_temp_Helsinki", "DIFF": "DIFF_Helsinki"}



In [10]:
type(new_names)

dict

In [13]:
merge_helsinki = join.rename(columns=new_names)
merge_helsinki.head()

,YEARMONTH,TEMP_C_Helsinki,MONTH,TAVG_Helsinki,ref_temp_Helsinki,DIFF_Helsinki
0,195201,-0.416667,1.0,21.154787,-6.025118,5.608452
1,195202,-3.638889,2.0,20.899016,-6.167213,2.528324
2,195203,-10.895062,3.0,27.792892,-2.337282,-8.557779
3,195204,-0.277778,4.0,38.546655,3.637031,-3.914809
4,195205,5.555556,5.0,50.077766,10.043203,-4.487648


In [18]:
merge_helsinki.to_csv("Desktop/SHK/DBAdmin/GeoPythonHelsinki/site/source/notebooks/L6/data/merge_helsinki.csv")

OSError: Cannot save file into a non-existent directory: 'Desktop\SHK\DBAdmin\GeoPythonHelsinki\site\source\notebooks\L6\data'